# Mini-Project Starter: Logistic Regression (Derive & Implement)

## 1. Load data and preprocess

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
p = Path('../04_datasets/logistic_data.csv')
try:
    df = pd.read_csv(p)
except Exception:
    # fallback synthetic
    df = pd.DataFrame({'x1':[0.5,1.0,1.5,2.0,2.5],'x2':[1.0,1.2,0.8,1.5,2.2],'y':[0,0,1,1,1]})
X = df[['x1','x2']].values
y = df['y'].values
X = np.hstack([np.ones((X.shape[0],1)), X])  # add intercept
print('X shape', X.shape)
print(df.head())

## 2. Define sigmoid and loss

In [ ]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

def neg_log_likelihood(w,X,y):
    z = X.dot(w)
    p = sigmoid(z)
    # small epsilon for numerical stability
    eps = 1e-12
    return -np.sum(y*np.log(p+eps) + (1-y)*np.log(1-p+eps))

w = np.zeros(X.shape[1])
print('initial loss', neg_log_likelihood(w,X,y))

## 3. Analytic gradient and Hessian (to implement)

In [ ]:
def grad_and_hessian(w,X,y):
    z = X.dot(w)
    p = sigmoid(z)
    grad = X.T.dot(p - y)
    # Hessian: X^T D X where D is diag(p*(1-p))
    D = np.diag(p*(1-p))
    H = X.T.dot(D).dot(X)
    return grad, H

print('grad,hess shapes', grad_and_hessian(w,X,y)[0].shape, grad_and_hessian(w,X,y)[1].shape)

## 4. Implement Newton's method & Gradient Descent templates (fill in)

In [ ]:
def gradient_descent(w0, X, y, lr=0.1, iters=1000):
    w = w0.copy()
    losses = []
    for i in range(iters):
        g,_ = grad_and_hessian(w,X,y)
        w = w - lr * g
        losses.append(neg_log_likelihood(w,X,y))
    return w, losses

def newtons_method(w0, X, y, iters=20):
    w = w0.copy()
    losses = []
    for i in range(iters):
        g,H = grad_and_hessian(w,X,y)
        # solve H p = g
        try:
            p = np.linalg.solve(H, g)
        except np.linalg.LinAlgError:
            p = np.linalg.pinv(H).dot(g)
        w = w - p
        losses.append(neg_log_likelihood(w,X,y))
    return w, losses

w0 = np.zeros(X.shape[1])
w_gd, losses_gd = gradient_descent(w0,X,y,lr=0.1,iters=200)
w_nm, losses_nm = newtons_method(w0,X,y,iters=20)
print('final gd loss', losses_gd[-1])
print('final newton loss', losses_nm[-1])

## 5. Plot convergence (students fill in)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses_gd,label='GD')
plt.plot(losses_nm,label='Newton')
plt.yscale('log')
plt.legend(); plt.xlabel('iter'); plt.ylabel('loss'); plt.show()